In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN,Dense

#Loading the Dataset

In [22]:
max_feat = 10000 #maximum features
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_feat)

##printing the shape of the data
print(f'Training data shape :{x_train.shape} , Training lables shape :{y_train.shape}')
print(f'Testing data shape :{x_test.shape} , Testing lables shape :{y_test.shape}')


Training data shape :(25000,) , Training lables shape :(25000,)
Testing data shape :(25000,) , Testing lables shape :(25000,)


Padding


In [23]:
x_train = sequence.pad_sequences(x_train,padding ='pre',maxlen = 500)
x_test = sequence.pad_sequences(x_test,padding ='pre',maxlen = 500)

In [24]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

# Train simple RNN

In [25]:
model = Sequential()
model.add(Embedding(max_feat,128,input_length= 500)) ##Embedding layer 
model.add(SimpleRNN(128,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1313025 (5.01 MB)
Trainable params: 1313025 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.compile('adam','binary_crossentropy',metrics = ['accuracy'])

# Creating an Instance of early stopping Callback

In [27]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights = True) ## Stop if no improvement for more than 5 Epochs

In [28]:
history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split = 0.3,callbacks = [earlystop])

Epoch 1/10
547/547 [==============================] - 65s 117ms/step - loss: 5279071.5000 - accuracy: 0.6087 - val_loss: 0.6182 - val_accuracy: 0.6416
Epoch 2/10
547/547 [==============================] - 63s 115ms/step - loss: 1.6063 - accuracy: 0.6770 - val_loss: 0.5943 - val_accuracy: 0.6633
Epoch 3/10
547/547 [==============================] - 63s 116ms/step - loss: 6417305.0000 - accuracy: 0.6137 - val_loss: 0.6459 - val_accuracy: 0.5951
Epoch 4/10
547/547 [==============================] - 63s 116ms/step - loss: 0.5978 - accuracy: 0.6714 - val_loss: 0.6367 - val_accuracy: 0.6135
Epoch 5/10
547/547 [==============================] - 67s 123ms/step - loss: 0.4992 - accuracy: 0.7461 - val_loss: 0.5079 - val_accuracy: 0.7599
Epoch 6/10
547/547 [==============================] - 87s 158ms/step - loss: 0.3720 - accuracy: 0.8342 - val_loss: 0.4848 - val_accuracy: 0.7903
Epoch 7/10
547/547 [==============================] - 79s 144ms/step - loss: 0.2793 - accuracy: 0.8853 - val_loss: 0.4

Save the model

In [29]:
model.save('simple_rnn_imdb.h5')